In [ ]:
#import potential packages needed

In [ ]:
from fredapi import Fred
fred = Fred(api_key='56e2cc23702c09f0c02226f2780c4de4') #censor this code eventually

## Table of Contents

1) Data Overview & Data Cleaning Process

# Section 1.1) Data Overview

1) First data point is a .txt file (~10GB) of Bond Data, downloaded from Wharton Research/Data Services (WRDS)
    * contains all bond trades between 2008 and 2017
    * Data included is bond ID, cusip_id, exact trade date/time, traded price, quoted_yield
    
2) Second data point is a .txt file from WRDS, relating to the corresponding data on bond maturities seniority etc.
    * contains bond ID, cusip_id, sub-product type, debt-type, issuer_name, maturity date, grade, convertible_flag, company_symbol
    
3) Third data point is a .txt file from WRDS, pertaining to coupon information per bond
    * contains: issue_id, coupon_type, offering_date, principal, first_interest_date, interest_frequency, coupon, day_count_basis, last_interest_date
    
4) Fourth data entry is information extracted from Fred, relating to information on swap_rates from one-year through to five-year (the risk-free rate proxy in this report)- all denonted in USD; in addition to values of VIX and other interest rate components

5) Fifth data entry is the trace-mergent linkfile
    * contains: bond_ID, issue_id, issue_id_fisd
    * This file is necessary to merge together the data about the bonds as WRDS has data on coupons, maturities, prices from different sub-vendors who potentially use different codes/reference numbers. This file will enable us to merge all of the above information on bonds and eventually, with their respective Credit Default Swap (CDS) counterparts. 
    
6) Sixth date entry is a .txt file (>20GB) of CDS trades on the same date range as the earlier bond information, from [DATASTREAM?] 
    * contains information on: series_id, gvkey, company_name, stock_ticker, source, duration, clause, currency, class_type, date, cds_spread
    
7) (By earlier filtered viable GVKEY sub-sample) Equivalent Equity trading data (Daily) from WRDS/Compustat (CSV ~1GB)
    * Includes: open, high, low, close, trading_volume......

8) Equity Sector Data & Returns (Daily-level) from WRDS/Compustat 

9) (By earlier filtered viable GVKEY sub-sample) Firm-level ratios
    * pertaining to: EPS....

10) (By earlier filtered viable GVKEY sub-sample) detailed breakdown of opening, high, low, closing bond-trades data

    

In [ ]:
#the try/except needs to be changed to the more recent format - with location etc. 


#pecds_1.30 version below 

In [ ]:
#bonds: Trade Day(date), bond_sym_id, price 
bond_file = open('sample_bond_data_all.txt','r')
bonds_date_price = {}

for line in bond_file:  
  line = line.rstrip("\r\n") 
  [bond_sym_id, date, trade_time, quantity, price, _yield] = line.split("\t")
  if bond_sym_id=="bond_sym_id" and date=='date':
    continue
  try:
      bonds_date_price[(date, bond_sym_id)] = price
  except ValueError:
    pass

try:
    bonds_date_price_df = pd.DataFrame(list(bonds_date_price.items()), columns=['date','date_id_price'])
except Exception:
    trackback.print_exc()
     
bond_file.close()

In [ ]:
#Create a datetime object for later merging this data with the relevant swap_rates / other data
format_date = '%Y%m%d'                                                                #American style date
datetime_obj = bonds_date_price_df['trade_date'].apply(lambda x: datetime.datetime.strptime(x, format_date))

#set the new datetime as DF index
bonds_date_price_df = bonds_date_price_df.set_index('trade_date')

In [ ]:
bond_file= open('master_file.txt','r')
bond_maturities = {}

for line in bond_file:  
  line = line.rstrip("\r\n") 
  [bond_sym_id, cusip_id, bsym_id, sub_prdct_type, debt_type_cd, issuer_nm, scrty_ds, cpn_rt, cpn_type_cd, trd_rpt_efctv_dt, mtrty_dt, grade, ind_144a, dissem, cnvrb_fl, company_symbol] = line.split("\t")
  if bond_sym_id=="bond_sym_id":
    continue
  try:
      bond_maturities[(bond_sym_id)] = mtrty_dt
  except ValueError:
    pass

try:
    bond_maturities_df = pd.DataFrame(list(bond_maturities.items()), columns=['bond_sym_id','maturity_date'])
except Exception:
    trackback.print_exc()
     
bond_file.close()

In [ ]:
#matched with the first file by bond_ID

In [ ]:
coupon_file= open('coupon_info.txt','r')
coupons_dict = {}

for line in coupon_file:  
  line = line.rstrip("\r\n")
  [ISSUE_ID, MATURITY, COUPON_TYPE, OFFERING_DATE, PRINCIPAL_AMT, FIRST_INTEREST_DATE,
   INTEREST_FREQUENCY, COUPON, DAY_COUNT_BASIS, LAST_INTEREST_DATE] = line.split("\t")
  if ISSUE_ID=="ISSUE_ID":
    continue
  try:
      coupons_dict[(ISSUE_ID, INTEREST_FREQUENCY, COUPON)] = ISSUE_ID, INTEREST_FREQUENCY, COUPON
  except ValueError:
    pass

try:
    coupons_df = pd.DataFrame(list(coupons_dict.items()), columns=['ISSUE_ID','INTEREST_FREQ'])
except RuntimeError as re:
    print("runtime error", re)
except Exception as other:
    print("something else", other)
 
coupon_file.close()

In [ ]:
#RFR DATA

In [ ]:
#INTERPOLATION OF RFRs

In [ ]:
#trace_mergent_linkfile.txt
merge_file= open('trace_mergent_linkfile.txt','r')
merge_dict = {}

for line in merge_file:  
  line = line.rstrip("\r\n")
  [bond_sym_id, issue_id_fisd] = line.split("\t")
  if bond_sym_id=="bond_sym_id":
    continue
  try:
      merge_dict[bond_sym_id, issue_id_fisd] = bond_sym_id, issue_id_fisd
  except ValueError:
    pass

try:
    mergefile_df = pd.DataFrame(list(merge_dict.items()), columns=['bond_sym_id','issue_id_fisd'])
except (RuntimeError, TypeError, NameError):
    print('ERROR')
     
merge_file.close()

In [ ]:
#CDS SPREAD

cds_file = open('sample_cds_data_all.txt','r')
cds_spread_dict = {}

for line in cds_file:  
  line = line.rstrip("\r\n") 
  [series_id, series_name, gvkey, company_name, ticker, source, duration, clause, currency, class_type, date, cds_spread] = line.split("\t")   
  if series_id=="series_id":
    continue
  if (float(duration)==5 and currency=="USD" and clause=="XR"):
    cds_spread_dict[gvkey, series_id, date] = cds_spread #date removed 
    
cds_file.close()

try: 
    cds_spread_df2 = pd.DataFrame(list(cds_spread_dict.items()),columns = ['cds_info','cds_spread']) 
except RuntimeError as re:
    print("runtime error", re)
except Exception as other:
    print("something else", other)